In [ ]:
#%cd /draco/ptmp/thay/orb5/marconi/iter_mm_nl_n20to30_g0_fulls
%cd /draco/ptmp/thay/orb5/marconi/iter_mm_nl_n20to30_nft2_mr200_np2x_N64_cl6
#%cd ../iter_mm_nl_n24to27_mr200_np1x
#%cd ../iter_mm_nl_n20to30_fulls_nft2_mr200_npe1B
#! orb5_utils.py -phi2d_use_n=1 -run_phi2d_peak=1 -update_npy=1 -phi_mmin=20 -phi_mmax=30
#! convert fft_modes_n_at_peaks.pdf fft_modes_n_at_peaks.png
# from IPython.display import Image
# fig = Image(filename=('fft_modes_n_at_peaks.png'))
# fig

In [ ]:
#!ls -d /draco/ptmp/thay/orb5/marconi/iter*

In [1]:
import h5py
import numpy as np
import orb5_utils
import orb5_3d_utils
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook

ModuleNotFoundError: No module named 'orb5_utils'

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
import plot_sig_noise

In [ ]:
orbp = orb5_3d_utils.orb5_3d_data()
orbp.load_hdf5()
orbp.load_phi3d()
orbp.load_nmodes()

In [ ]:
print(orbp.n_mode_numbers)
print(orbp.n_mode_numbers_nft)

In [ ]:
# sum on m-modes
n_ffts = orbp.n_fft()

In [ ]:
# plot field: sum on m, at every n: (t,s)
fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,4))
for intor, ntor in enumerate(orbp.n_mode_numbers):
    if ntor not in orbp.n_mode_numbers_nft:
        continue
    ax1.plot(orbp.p3t[:], np.max( np.abs(n_ffts[:,intor,:]), axis=1), label=ntor)
    ax2.plot(orbp.p3t[:], np.max(np.real(n_ffts[:,intor,:]), axis=1), label=ntor)
#    ax3.plot(orbp.p3t[:], np.max(np.imag(n_ffts[:,intor,:]), axis=1), label=ntor)
for axi in [ax1,ax2]:
    axi.set_yscale('log')
    axi.legend(framealpha=0.5, fancybox=True, loc='best', ncol=2)

In [ ]:
tstride = 50
fig, ax = plt.subplots()
for intor, ntor in enumerate(orbp.n_mode_numbers):
    if ntor not in orbp.n_mode_numbers_nft:
        continue
    ax.plot(orbp.p3t[::tstride], np.max(np.abs(n_ffts[::tstride,intor,:]), axis=1), label=ntor)
ax.set_yscale('log')
ax.legend(framealpha=0.5, fancybox=True, loc='best', ncol=2)

In [ ]:
orbp.p3dz.shape

In [ ]:
orbp.set_radial_offsets()
orbp.radial_offsets
orbp.load_serial()

In [ ]:
itime = -1
pscs = {}
pscs['itime'] = itime
pscs[-1] = orbp.create_potsc(itime)
for intor, ntor in enumerate(orbp.n_mode_numbers_nft):
    psc = orbp.create_potsc(itime, nchi=None, nlist=[ntor])
    pscs[ntor] = psc

In [ ]:
try:
    pscs[30].shape
except:
    pass

In [ ]:
fig, ax = plt.subplots(figsize=(4,7))
_ = orbp.plot_psc(np.real(pscs[-1]), outline=True, fig=fig, ax=ax, aspect=1)

In [ ]:
@interact(ntor=(min(orbp.n_mode_numbers), max(orbp.n_mode_numbers), orbp.nft), all_modes=False, outline_eq=False)
def plot_plane(ntor=-1, all_modes=False, outline_eq=False):
    fig, ax = plt.subplots(figsize=(4,7))
    moden = -1 if all_modes else ntor
    _ = orbp.plot_psc(np.real(pscs[moden]), fig=fig, ax=ax, aspect=1, cmap='RdBu_r', sym_clim=True, outline=True, outline_eq=outline_eq)


In [ ]:
precompute_pscs = False
itevery = 10
if precompute_pscs:
    try:
        print(pscs_n_t.keys())
    except:
        pscs_n_t = {}
    
    for ntor in orbp.n_mode_numbers_nft+[-1]:
        print(ntor)
        if ntor in pscs_n_t:
            continue
        pscs_n_t[ntor] = {}
        pscs_n_t[ntor]['n'] = ntor
        pscs_n_t[ntor]['times'] = {}
        pscs_n_t[ntor]['itimes'] = []
        for itime, time in enumerate(orbp.p3t[:]):
            if itime % itevery != 0:
                continue
            pscs_n_t[ntor]['itimes'].append(itime)
            pscs_n_t[ntor]['times'][itime] = time
            if ntor == -1:
                nlist = list(np.arange(20,30+1))
            else:
                nlist = [ntor]
            pscs_n_t[ntor][itime] = np.real(orbp.create_potsc(itime, nchi=None, nlist=nlist))

In [ ]:
if precompute_pscs:
    for key in pscs_n_t.keys():
        print(key, len(list(pscs_n_t[key].keys())))

In [ ]:
sub_kwargs={"figsize":(10,10)}

In [ ]:
if precompute_pscs:
    nrange=(min(orbp.n_mode_numbers),max(orbp.n_mode_numbers),orbp.nft)
    @interact(ntor=nrange,itime=(0,orbp.p3t.shape[0]-1),savefig=False, outline_eq=False, all_modes=False)
    def plot_pscs_t(ntor=None, itime=None, savefig=None, outline_eq=False, all_modes=False):
        if ntor not in pscs_n_t:
            return
        if itime not in pscs_n_t[ntor]['itimes']:
            itime = pscs_n_t[ntor]['itimes'][np.argmin(np.abs(np.array(pscs_n_t[ntor]['itimes']) - itime))]
        fig, ax = plt.subplots(**sub_kwargs)
        if all_modes:
            ntor = -1
        _ = orbp.plot_psc(np.real(pscs_n_t[ntor][itime]), fig=fig, ax=ax, aspect=1, cmap='RdBu_r', sym_clim=True, outline=True, outline_eq=outline_eq)
        ntitle = str(ntor) if ntor != -1 else "all"
        ax.set_title('n={}, t={}'.format(ntitle, pscs_n_t[ntor]['times'][itime]))
        if savefig:
            fig_filename='psc_n{}_at_t{}.png'.format(ntitle, pscs_n_t[ntor]['times'][itime])
            fig.savefig(fig_filename)
            print("fig saved to {}".format(fig_filename))

In [ ]:
pscs_on_the_fly = False
if pscs_on_the_fly:
    nrange=(min(orbp.n_mode_numbers),max(orbp.n_mode_numbers),orbp.nft)
    @interact(ntor=nrange,itime=(0,orbp.p3t.shape[0]-1),savefig=False, outline_eq=False, all_modes=False)
    def plot_pscs_t_on_fly(ntor=None, itime=None, savefig=None, outline_eq=False, all_modes=False):
        if ntor not in orbp.n_mode_numbers_nft:
            return
        fig, ax = plt.subplots(**sub_kwargs)
        if all_modes:
            nlist = orbp.n_mode_numbers_nft
            ntitle = 'all'
        else:
            nlist = [ntor]
            ntitle = str(ntor)
        psc_arr = np.real(orbp.create_potsc(itime, nchi=None, nlist=nlist))
        _ = orbp.plot_psc(psc_arr, fig=fig, ax=ax, aspect=1, cmap='RdBu_r', sym_clim=True, outline=True, outline_eq=outline_eq)
        ax.set_title('n={}, t={}'.format(ntitle, orbp.p3t[itime]))
        if savefig:
            fig_filename='psc_n{}_at_t{}.png'.format(ntitle, orbp.p3t[itime])
            fig.savefig(fig_filename)
            print("fig saved to {}".format(fig_filename))

In [ ]:
nrange=(min(orbp.n_mode_numbers), max(orbp.n_mode_numbers), orbp.nft)
itrange=widgets.IntSlider(min=0, max=orbp.p3t.shape[0]-1, value=orbp.p3t.shape[0]-1)
@interact_manual(ntor=nrange,itime=itrange,savefig=False, outline_eq=False, all_modes=False)
def plot_pscs_t_on_fly(ntor=None, itime=None, savefig=None, outline_eq=False, all_modes=False):
    if ntor not in orbp.n_mode_numbers_nft:
        return
    fig, ax = plt.subplots(**sub_kwargs)
    if all_modes:
        ntitle="all"
        nlist = orbp.n_mode_numbers_nft
    else:
        ntitle=str(ntor)
        nlist = [ntor]
    psc_arr = np.real(orbp.create_potsc(itime, nchi=None, nlist=nlist))
    _ = orbp.plot_psc(psc_arr, fig=fig, ax=ax, aspect=1, cmap='RdBu_r', sym_clim=True, outline=True, outline_eq=outline_eq)
#    ntitle = str(ntor) if ntor != -1 else "all"
    ax.set_title('n={}, t={}'.format(ntitle, orbp.p3t[itime]))
    if savefig:
        fig_filename='psc_n{}_at_t{}.png'.format(ntitle, orbp.p3t[itime])
        fig.savefig(fig_filename)
        print("fig saved to {}".format(fig_filename))

In [ ]:
mlist=None
mlsb=2
musb=5
nrange=(min(orbp.n_mode_numbers),max(orbp.n_mode_numbers),orbp.nft)
@interact(ntor=nrange,savefig=False, save_all=False)
def pfn_obj(ntor=None, savefig=None, save_all=False):
    if save_all:
        for ntor in orbp.n_mode_numbers_nft:
            mlist = [ntor+i for i in np.arange(-mlsb,+musb+1)]
            orbp.plot_fft_modes_at_peaks(ntor=ntor, savefig=True, mlist=mlist)
    else:
        mlist = [ntor+i for i in np.arange(-mlsb,+musb+1)]
        orbp.plot_fft_modes_at_peaks(ntor=ntor, savefig=savefig, mlist=mlist)

In [ ]:
"""
Caveat w.r.t. radial coordinate
"""
mlist=None
nrange=(min(orbp.n_mode_numbers),max(orbp.n_mode_numbers),orbp.nft)
sfmin=orbp.orb5s.h5['/parameters/fields/sfmin'][0]
sfmax=orbp.orb5s.h5['/parameters/fields/sfmax'][0]
sfrange=widgets.FloatSlider(min=sfmin,max=sfmax,step=0.01,value=0.5)
@interact(ntor=nrange, spos=sfrange, savefig=False, save_all=False)
def pfns_obj(ntor=None, savefig=None, spos=None, save_all=False):
    if save_all:
        for ntor in orbp.n_mode_numbers_nft:
            mlist = [ntor+i for i in np.arange(-mlsb,+musb+1)]
            orbp.plot_fft_modes_at_spos(ntor=ntor, spos=spos, savefig=True, mlist=mlist)
    else:
        mlist = [ntor+i for i in np.arange(-mlsb,+musb+1)]
        orbp.plot_fft_modes_at_spos(ntor=ntor, spos=spos, savefig=savefig, mlist=mlist)

In [ ]:
def plot_m_of_n_at_t(ntor=None, imode=None, itime=None, time=1E6, savefig=False, full_range=True):
    fig, ax = plt.subplots()
    if itime is None:
        itime = np.argmin(np.abs(orbp.p3t[:] - time))
    if imode is None:
        imode, = np.arange(len(orbp.n_mode_numbers))[np.array(orbp.n_mode_numbers)==ntor]
    if full_range:
        mmin = orbp.mharms[imode].min()
        mmax = orbp.mharms[imode].max()
    else:
        mmin = ntor - mlsb
        mmax = ntor + musb
    for iharm, mharm in enumerate(np.arange(mmin, mmax+1)):
        harm = np.sum(orbp.p3dz[itime,imode]*(orbp.mharms[imode]==mharm), axis=1)
        ax.plot(np.abs(harm), label=mharm)
    ax.legend(loc='best', framealpha=0.5, ncol=2)
    ax.set_title("t={}".format(orbp.p3t[itime]))
    if savefig:
        fig.savefig('mharms_in{}_it{}.pdf'.format(imode, itime))
        
nrange=(min(orbp.n_mode_numbers),max(orbp.n_mode_numbers),orbp.nft)
itrange=widgets.IntSlider(min=0, max=orbp.p3t.shape[0]-1, value=orbp.p3t.shape[0]-1, continuous_update=False)
@interact(ntor=nrange, itime=itrange, savefig=False, full_range=False)
def plot_m_of_n_at_t_jup(ntor=None, itime=None, savefig=False, full_range=False):
    return plot_m_of_n_at_t(ntor=ntor, itime=itime, savefig=savefig, full_range=full_range)

In [ ]:
nrange=(min(orbp.n_mode_numbers),max(orbp.n_mode_numbers),orbp.nft)
mrange=widgets.IntSlider(min=0,max=100,value=25)
@interact_manual(mharm=mrange, ntor=nrange, imode=fixed(None))
def m_s_of_t(mharm=-1, imode=None, ntor=None):
    if imode is None:
        imode, = np.arange(len(orbp.n_mode_numbers))[np.array(orbp.n_mode_numbers)==ntor]
    mamp = np.zeros((orbp.p3t.shape[0], orbp.ns_), dtype=np.complex64)
    tmp = orbp.mharms[imode]
    if mharm > tmp.min() and mharm < tmp.max():
        for itime, time in enumerate(orbp.p3t[:]):
            mamp[itime,:] = np.sum(orbp.p3dz[itime,imode]*(tmp==mharm), axis=1)
    st_grid = np.meshgrid(np.arange(orbp.ns_), orbp.p3t[:])
    im = plt.pcolormesh(st_grid[0], st_grid[1], np.real(mamp), cmap='RdBu_r')
    im.set_clim(np.array([-1,1])*np.max(np.abs(im.get_clim())))

In [ ]:
fig, ax = plot_sig_noise.plot_sig_noise()

In [ ]:
orb = orb5_utils.orb5_data()
orb.load_hdf5()
orb.load_phi(phi_2d_type='potsp')
p2pt = orb.phi_2d['time']
@interact(itime=widgets.IntSlider(min=0, max=len(p2pt[:])-1, values=len(p2pt[:]-1), continuous_update=False))
def plot_n_of_s_at_t(itime=-1):
    fig, ax = orb.plot_fft_modes_at_t(itime, ifreqlist=orbp.n_mode_numbers_nft)
    ax.set_title("t={}".format(p2pt[itime]));

In [ ]:
print('Miscellaneous extra')

In [ ]:
a  = orbp.get_m_imode(0,15)

In [ ]:
im = plt.pcolormesh(orbp.p2cr, orbp.p2cz, np.real(pscs[-1]), cmap='RdBu_r')
im.set_clim(np.array([-1,1])*np.max(np.abs(im.get_clim())))
plt.gca().set_aspect(1)

In [ ]:
pscs[-1].shape

In [ ]:
@interact(a=(0,100,2))
def testprint(a=1):
    print(a)

In [ ]:
fp = h5py.File('orb5_res_parallel.h5', 'r')

In [ ]:
p3 = fp['/data/var3d/generic/pot3d/']

In [ ]:
p3d = p3['data']
p3t = p3['time']
p3m = p3['mmin']

In [ ]:
# p3dz = (p3d['real'] + 1j*p3d['imaginary'])
p3dz = p3d

In [ ]:
#orb = orb5_utils.orb5_data()
#orb.load_hdf5()
orb = orbp.orb5s

In [ ]:
p2c = orb.h5['/data/var2d/generic/potsc']
p2cd = p2c['data']
p2ct = p2c['time']
p2cr = p2c['rsc']
p2cz = p2c['zsc']
p2cs = p2c['coord1']
p2cc = p2c['coord2']

In [ ]:
p2p = orb.h5['/data/var2d/generic/potsp']
p2pd = p2p['data']
p2pt = p2p['time']
p2ps = p2p['coord1']
p2pp = p2p['coord2']

In [ ]:
plt.pcolormesh(np.abs(p3dz[-1,-1,:,:]))

In [ ]:
itime = -1
fig, ax = plt.subplots(figsize=(4,6))
im = ax.pcolormesh(p2cr, p2cz, p2cd[itime], cmap='RdBu_r')
im.set_clim(np.array([-1,1])*np.max(np.abs(im.get_clim())))
ax.plot(p2cr[:,0], p2cz[:,0], color='k')
ax.plot(p2cr[:,-1], p2cz[:,-1], color='k')
ax.plot(orb.h5['/equil/sc/r_c'][:,-1], orb.h5['/equil/sc/z_c'][:,-1])
ax.set_aspect('equal')
ax.set_title("t={}".format(p2ct[itime]));

In [ ]:
plt.plot(orb.h5['/equil/sc/r_c'][:,-1], orb.h5['/equil/sc/z_c'][:,-1])

In [ ]:
np.abs(p3m)

In [ ]:
p3d.shape

In [ ]:
a = np.meshgrid(np.arange(p3d.shape[3]), np.arange(p3d.shape[2]))

In [ ]:
b = a[0]

In [ ]:
b.shape

In [ ]:
p3m.shape

In [ ]:
full_range = True
savefig = False
#a = np.meshgrid(np.arange(p3d.shape[3]), np.arange(p3d.shape[2]))
#b = a[0]
for imode in range(p3m.shape[0]):
    if orbp.n_mode_numbers[imode] not in orbp.n_mode_numbers_nft:
        continue
    ntor = orbp.n_mode_numbers[imode]
    fig, ax = plt.subplots()
    itime = -1
    time = p3t[itime]
    #tmp = np.abs(np.array([p3m[imode]]).transpose() + b)
    tmp = orbp.mharms[imode]
    if full_range:
        mmin = tmp.min()
        mmax = tmp.max()
    else:
        mmin = 5
        mmax = 12
    for iharm, mharm in enumerate(np.arange(mmin, mmax+1)):
        harm = np.sum(p3dz[itime,imode]*(tmp==mharm), axis=1)
        ax.plot(np.abs(harm), label=mharm)
    ax.legend(loc='best', framealpha=0.5, ncol=2)
    ax.set_title("n={}, t={}".format(ntor, time))
    if savefig:
        fig.savefig('diag_3drhsf_in{}_it{}.pdf'.format(imode, itime))

In [ ]:
p3m

In [ ]:
a = np.meshgrid(np.arange(p3d.shape[3]), np.arange(p3d.shape[2]))
b = a[0]
fig, axarr = plt.subplots(2, sharex=True)
imode = -1
itime = -1
ntor = orbp.n_mode_numbers[imode]
for iharm, mharm in enumerate(ntor + np.arange(-mlsb,musb+1)):
    tmp1 = np.abs(p3m[:]).transpose() - b
    tmp2 = orbp.mharms[imode]
    for itmp, tmp in enumerate([tmp1,tmp2]):
        harm = np.sum(p3dz[itime,imode]*(tmp==mharm), axis=1)
        axarr[itmp].plot(np.abs(harm), label=mharm)
        ax.legend(loc='best', framealpha=0.5)

In [ ]:
plt.pcolormesh(np.abs(p3dz[itime,imode]*(tmp==mharm)))

In [ ]:
b.shape

In [ ]:
p3dz.shape

In [ ]:
#p3dz[-1,-1][b == 5]

In [ ]:
tmp = np.abs(p3m[:]).transpose() + b
b[tmp == 15]

In [ ]:
tmp = np.abs(p3m[:]).transpose() - b
test = b[b[tmp == 9]]

In [ ]:
np.sum(b * (tmp == 30), axis=1)

In [ ]:
plt.plot(np.sum(np.abs(p3dz[-1,-1]) * (tmp == 9), axis=1))

In [ ]:
mharm = 31
imode = -1
tmp = orbp.mharms[imode]
fig, axarr = plt.subplots(1,3, sharey=True)
fig2, ax = plt.subplots()
axarr[0].pcolormesh(np.abs(p3dz[-1,imode]))
axarr[1].pcolormesh((tmp == mharm))
axarr[2].pcolormesh(np.abs(p3dz[-1,imode]) * (tmp==mharm))
ax.plot(np.sum(np.abs(p3dz[-1,imode]) * (tmp==mharm), axis=1), linestyle='-')
ax.plot(np.sum(np.abs(p3dz[-1,imode]) * (tmp==mharm+1), axis=1), linestyle='--')
ax.plot(np.sum(np.abs(p3dz[-1,imode]) * (tmp==mharm+-1), axis=1), linestyle='--')
#fig.tight_layout()
#fig.savefig('m{}_summary.pdf'.format(mharm))

In [ ]:
plt.pcolormesh(tmp-25)

In [ ]:
p2cd.shape

In [ ]:
def test_psp(nlist, potsc_c, p3dz_single, sgrid, chigrid, deltam, mharmlist):
    jj = 1j
    ns = len(sgrid)
    for imode in range(len(nlist)):
        ntor = nlist[imode]
        for ispos in range(ns):
            for im in range(2*deltam+1):
                mharm = mharmlist[imode, ispos, im]
                for ichi in range(len(chigrid)):
                    chi = chigrid[ichi]
                    potsc_c[ichi,ispos] = potsc_c[ichi,ispos] + p3dz_single[imode, ispos, im] * np.exp(jj * mharm * chi)


In [ ]:
def test_psp2(nlist, potsc_c, p3dz_single, sgrid, chigrid, deltam, mharmlist):
    ns = len(sgrid)
    for imode in range(len(nlist)):
        ntor = nlist[imode]
        for ispos in range(ns):
            for im in range(2*deltam+1):
                mharm = mharmlist[imode,ispos,im]
                potsc_c[:,ispos] = potsc_c[:,ispos] + p3dz_single[imode, ispos, im] * np.exp(1j * mharm * chigrid)


In [ ]:
orbp.mharms.shape

In [ ]:
from numba import jit, njit
test_psp_jit = jit(test_psp)
test_psp2_jit = jit(test_psp2)

nchi = p2cc[:].shape[0]-1
chigrid = 2*np.pi*(np.arange(nchi+1) / nchi)
sgrid = p2cs[:]

chigrid = p2cc[:]

ns = sgrid.shape[0]

p3dz_single = p3dz[-1]
nlist = orbp.n_mode_numbers_nft[:]
potsc_c = np.zeros((nchi+1, ns), dtype=np.complex64)

%timeit(test_psp_jit(nlist, potsc_c, p3dz_single, sgrid, chigrid, 5, orbp.mharms[:]))
#%timeit(test_psp(nlist, potsc_c, p3dz_single, sgrid, chigrid, 5, orbp.mharms[:]))
%timeit(test_psp2_jit(nlist, potsc_c, p3dz_single, sgrid, chigrid, 5, orbp.mharms[:]))
%timeit(test_psp2(nlist, potsc_c, p3dz_single, sgrid, chigrid, 5, orbp.mharms[:]))

In [ ]:
p3dz_single.shape

In [ ]:
(chigrid).shape

In [ ]:
ispos = 30
time = 1.E6
itime_2d = np.argmin(np.abs(p2ct[:] - time))
time_2d = p2ct[itime_2d]
itime_3d = np.argmin(np.abs(p3t[:] - time_2d))

ns = p3dz.shape[2]-2
nchi = p2cc.shape[0]-1

sgrid = p2cs
chigrid = 2*np.pi*(np.arange(nchi+1) / nchi)
potsc_c = np.zeros((nchi+1, ns), dtype=np.complex64)
for imode, ntor in enumerate(orbp.n_mode_numbers):
    if ntor not in orbp.n_mode_numbers_nft:
        continue
    
    for ispos in range(ns):
        for im in np.arange(p3dz.shape[3]):
#            mharm = tmp[ispos, im]
            mharm = orbp.mharms[imode][ispos,im]
            potsc_c[:,ispos] = potsc_c[:,ispos] + (p3dz[itime_3d, imode, ispos, im] * np.exp(1j * mharm * chigrid))

fig, axarr = plt.subplots(1,3, figsize=(30,10))
axarr[0].pcolormesh(p2cr, p2cz, np.real(potsc_c))
axarr[0].set_aspect(1)
axarr[0].set_title("t={} from 3D (real), code above".format(p3t[itime_3d]));

axarr[1].pcolormesh(p2cr, p2cz, p2cd[itime_2d])
axarr[1].set_aspect('equal')
axarr[1].set_title("t={} from 2D".format(p2ct[itime_2d]));

#axarr[2].pcolormesh(p2cr, p2cz, np.imag(potsc_c))
#axarr[2].set_aspect(1)
#axarr[2].set_title("t={} from 3D (imag)".format(p3t[itime_3d]));


psc = np.real(orbp.create_potsc(itime_3d))
axarr[2].pcolormesh(p2cr, p2cz, psc)
axarr[2].set_aspect(1)
axarr[2].set_title("t={} from 3D (real), using function".format(p3t[itime_3d]));

In [ ]:
chigrid.shape

In [ ]:
print(p2cd[itime_2d].shape)
print(potsc_c.shape)

In [ ]:
time = 1.e6
itime_2d = np.argmin(np.abs(p2ct[:] - time))
time_2d = p2ct[itime_2d]
itime_3d = np.argmin(np.abs(p3t[:] - time_2d))
time_3d = p3t[itime_3d]
print(time_2d, time_3d)
itime = itime_3d
chi_ref = 0.0
ichi = np.argmin(np.abs(p2cc[:] - chi_ref))

fig, axarr = plt.subplots(1,3, figsize=(10,3))

axarr[0].plot(np.real(potsc_c[ichi,:]))
axarr[0].plot(np.imag(potsc_c[ichi,:]), '--')
axarr[0].plot(np.abs( potsc_c[ichi,:]), 'g-')

ns = p3dz.shape[2]-2
print(p2ps.shape, p2pp.shape, p2pd.shape, p2pt.shape)
nphi = p2pp.shape[0]


if chi_ref == 0.0:
    potsc_c_slice = np.sum(p3dz[itime,:,:-2,:], axis=2)
else:
    potsc_c_slice = potsc_c[ichi,:-2]

sgrid = p2cs
phigrid = 2*np.pi*(np.arange(nphi) / nphi)
potsp = np.zeros((nphi, ns), dtype=np.complex64)
for imode, ntor in enumerate(orbp.n_mode_numbers):
    if ntor not in orbp.n_mode_numbers_nft:
        continue
    for ispos in range(ns):
        potsp[:,ispos] = potsp[:,ispos] + (potsc_c_slice[imode, ispos]* np.exp(-1j * ntor * phigrid))
    
axarr[1].pcolormesh(np.real(potsp[:,:]))
axarr[2].pcolormesh(p2pd[itime_2d, :,:])
#axarr[2].pcolormesh(p2pd[itime, :,:] - np.real(potsp[:,:]))
print(np.max(np.abs(np.real(potsp[:,:]))))
print(np.max(np.abs(p2pd[itime_2d, :,:])))

print(np.max(np.abs(np.real(potsp[:,:]))) / np.max(np.abs(p2pd[itime_2d, :,:])))
print(np.max(np.abs(p2pd[itime_2d, :,:] - np.real(potsp[:,:]))))

In [ ]:
#dummy = np.ones((ns), dtype=np.complex64)
dummy = np.sum(p3dz[itime,imode,:-3,:], axis=1)
plt.plot(np.real(dummy))
plt.plot(np.imag(dummy), '--')

"""
for im in np.arange(p3dz.shape[3]):
    mharm = tmp[ispos, im]
    potsc_c[:,ispos] = potsc_c[:,ispos] + (p3dz[itime, imode, ispos, im] * np.exp(1j * mharm * chigrid))
"""

In [ ]:
plt.pcolormesh(p2cr, p2cz, np.angle(potsc_c))

In [ ]:
smesh = np.meshgrid(np.arange(tmp.shape[1]), np.arange(tmp.shape[0]))[1]

In [ ]:
print(smesh.shape, tmp.shape, p3dz[-1,-1].shape)

In [ ]:
plt.pcolormesh(smesh, tmp, np.abs(p3dz[-1,-1]), shading='flat')

In [ ]:
itime = -1
sfmin = 0.2
sfmax = 0.7
deltas = (sfmax - sfmin) / (tmp.shape[0]-3)

dense_mmesh = np.meshgrid(np.arange(tmp.shape[0]), np.arange(tmp.max()+1))
dense_p3dz_t1 = np.zeros(dense_mmesh[0].shape, dtype=np.complex64)

mwidth = tmp.shape[1]
for ispos in np.arange(tmp.shape[0]):
    for imharm, mharm in enumerate(tmp[ispos]):
        dense_p3dz_t1[mharm,ispos] = p3dz[itime, imode, ispos, imharm]
plt.pcolormesh(sfmin + deltas * dense_mmesh[0], dense_mmesh[1] - 0.5, np.abs(dense_p3dz_t1))
plt.plot(sfmin + deltas * dense_mmesh[0][0,:], np.abs(p3m[0])-0.5)
plt.plot(sfmin + deltas * dense_mmesh[0][0,:], np.abs(p3m[0])-mwidth-0.5)
plt.plot(sfmin + deltas * dense_mmesh[0][0,:], dense_mmesh[0][0,:]*0+7)

In [ ]:
p3m[:]